In [77]:
import pandas as pd
import numpy as np
import csv
import gc
import os
import copy
from joblib import Parallel, delayed
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
#from sklearn.model_selection import train_test_split
from random import randint
from bisect import bisect
from math import log, e
from sklearn.feature_selection import mutual_info_classif as mi_clasf

In [ ]:
# loading preselected nodes
main_directory= '/home/a20114261/alarm_datasets/Alarm10/'
save_folder='/home/a20114261/alarm_datasets/log_results/'

if not os.path.exists(main_directory):
    print("Bad routing.")
preselected_nodes=[] # will have [node_str,balance_ranking]
# so far, we need Node, size of estimated pc set per filename, balance ranking
ff = open(save_folder+'mmpc_partitioned/alarm10_binary_nodes_selected.txt','r')
for e in ff.read().split('\n'):
    spl_line = e.split(';')
    if (len(spl_line[0])==0):
        continue
    preselected_nodes.append([spl_line[0].split('@')[0],float(spl_line[1])])
ff.close()

In [50]:
node_pc_size=[]
ff = open(save_folder+'mmpc_partitioned/alarm10_binary_nodes_pc_size.txt','r')
for e in ff.read().split('\n'):
    spl_line = e.split(';')
    if (len(spl_line[0])==0):
        continue
    node_pc_size.append([spl_line[0],spl_line[1],int(spl_line[2])])
ff.close()    

In [ ]:
graph_heads = []
for i in range(1,371):
    graph_heads.append('Node'+str(i))

In [ ]:
mi_score = mi_clasf(X, X[X.keys()[1]],discrete_features=True)
print(len(mi_score))
for e in mi_score:
    print(e)

In [ ]:
%%time
alpha=0.05
print("======================================================================================")
print("Entropy-Based Mutual Information Evaluation")
print("======================================================================================")


pre_PCs=[]
for filename in os.listdir(main_directory):
    for node in [x[0] for x in preselected_nodes]:
        PobDict={}
        PobDict['TargetNode']=node
        PobDict['filename']=filename
        pre_PCs.append(PobDict)


PCs = Parallel(n_jobs=40)(map(delayed(EntropyBasedMI),pre_PCs))

In [74]:
from operator import itemgetter
prescored_PCs=[]
for e in PCs:
    pc_length = [x[2] for x in node_pc_size if x[0]==e['filename'] and x[1].split('@')[0]==e['TargetNode']]
    if (len(pc_length)==0):
        e['PC']=[]
        continue
    e['PC']= [x[0] for x in sorted(e['evaluated_nodes'],key=itemgetter(1),reverse=True)[:pc_length[0]]]
    

In [ ]:
%%capture
# evaluation for each node, using each file as training set, in parallel
print("======================================================================================")
print("Entropy-Based Mutual Information Scoring phase")
print("======================================================================================")

PC_mean_score = Parallel(n_jobs=40)(map(delayed(CandidateScore_Parallel),PCs))

3899

In [82]:
log_directory="/home/a20114261/alarm_datasets/log_results/mmpc_partitioned"
ff = open(log_directory+'/scored_entropy-mi_log.txt','w')
for e in sorted(PC_mean_score, key=itemgetter('filename'),reverse=True):
    ff.write(e['filename']+';')
    ff.write(e['TargetNode']+';')
    for node in e['PC']:
        ff.write(node+'_')
    ff.write(';')
    for class_pred in e['class_precisions']:
            ff.write(class_pred[1]+'_')
            for arr_acc in class_pred[0]:
                ff.write(str(arr_acc[0])+'\t')
                ff.write(str(arr_acc[1])+'\t')
                ff.write(str(arr_acc[2])+'\t')
            ff.write('_')
    ff.write('\n')
ff.close()

In [ ]:
def EntropyBasedMI(PobDict):
    X = pd.read_csv(main_directory+'/'+filename,delimiter='  ',header=None)
    X.columns = graph_heads
    mi_score = mi_clasf(X, X[PobDict['TargetNode']],discrete_features=True)
    PobDict['evaluated_nodes']=[]
    for i in range(0,len(mi_score)):
        if i == (int(PobDict['TargetNode'].split('Node')[1])-1):
            continue
        PobDict['evaluated_nodes'].append(['Node'+str(i+1),mi_score[i]])
    print('Exiting Entropy-based Mutual Information analysis for '+PobDict['TargetNode']+' with file '+PobDict['filename'])
    return PobDict

In [68]:
def CandidateScore_Parallel(PobDict):
    global x_heads_train

    # training and scoring
    X_train_df = pd.read_csv(main_directory+PobDict['filename'],delimiter='  ',header=None)
    X_train_df.columns=graph_heads
    
    #X_train, X_test, Y_train, Y_test = train_test_split(X_train_df, X_train_df[TargetEvalNode], test_size=0.1, random_state=0)
    
    X_train = X_train_df
    Y_train = X_train_df[PobDict['TargetNode']]
    
    clf = LinearSVC()
    clf.fit(X_train[PobDict['PC']],Y_train)
    
    PobDict['class_precisions']=[]
    for filename in [x for x in os.listdir(main_directory) if x.split('_')[1]==PobDict['filename'].split('_')[1]]:
        print(len([x for x in os.listdir(main_directory) if x.split('_')[1]==PobDict['filename'].split('_')[1]]))
        if (filename == PobDict['filename']):
            continue
        print("*****")
        print()
        print("Testing on dataset: "+filename)
        X_test = pd.read_csv(main_directory+filename,delimiter='  ',header=None)
        X_test.columns = graph_heads
        

        # get score values
        precision = []
        values_counter=set(X_train_df[PobDict['TargetNode']])
        for val in values_counter:
            #Y_pred_c = clf.predict(X_train_df.query( TargetEvalNode+'== '+str(i))[PobDict['supercpc']][-100:])
            if (len(X_test.query( PobDict['TargetNode']+'== '+str(val)))==0):
                precision.append([val,0,0])
                continue
            Y_pred_c = clf.predict(X_test.query( PobDict['TargetNode']+'== '+str(val))[PobDict['PC']])
            precision.append([val,len(Y_pred_c),Y_pred_c.tolist().count(val)/len(Y_pred_c)])

        for e in range(0,len(precision)):
            print('precision for class '+str(precision[e][0])+' with '+str(precision[e][1])+' samples in dataset : '+str(precision[e][2]))
    
        print("G-mean score: "+str(g_mean(precision)))
        print("*****")
        print()
        #print(clf.score(X_test[PobDict['supercpc'][:10]],Y_test))
        PobDict['class_precisions'].append([precision,filename])
    return PobDict

In [69]:
def balance_scoring(size1, size2):
    f_size1 = float(size1)
    f_size2 = float(size2)
    if (f_size1/f_size2 <= 1):
        return f_size1/f_size2
    else:
        return f_size2/f_size1

In [70]:
def g_mean(precision_arr):
    counter=1
    for e in precision_arr:
        counter=counter*e[2]
    
    return counter**(1/len(precision_arr))